In [2]:
%pip install ffmpeg-python

%apt-get update
%apt-get install -y ffmpeg

Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%apt-get` not found.


In [3]:
import whisper
import ffmpeg
from transformers import MarianMTModel, MarianTokenizer

def extract_audio(video_file, audio_file):
    """Extracts audio from video."""
    ffmpeg.input(video_file).output(audio_file).run(overwrite_output=True)

def transcribe_audio(audio_file, language):
    """Transcribes audio to text using Whisper in the specified language."""
    model = whisper.load_model("base")
    result = model.transcribe(audio_file, language=language)  # Use the specified language
    return result['segments']

def format_time(seconds):
    """Formats time from seconds to SRT format (hh:mm:ss,ms)."""
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = int((seconds - int(seconds)) * 1000)
    return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02},{milliseconds:03}"

def translate_text_marian(text, model, tokenizer):
    """Translates text using MarianMT."""
    translated = model.generate(**tokenizer.prepare_seq2seq_batch([text], return_tensors="pt"))
    return tokenizer.decode(translated[0], skip_special_tokens=True)

def generate_srt(segments, srt_file, model=None, tokenizer=None):
    """Generates an SRT file from transcription segments.

    If model and tokenizer are None, the subtitles will be in the original language of the video.
    """
    with open(srt_file, "w") as file:
        for i, segment in enumerate(segments):
            start_time = format_time(segment['start'])
            end_time = format_time(segment['end'])
            text = segment['text'].strip()

            if model and tokenizer:
                text = translate_text_marian(text, model, tokenizer)

            file.write(f"{i + 1}\n{start_time} --> {end_time}\n{text}\n\n")

def main(video_file, output_srt, language, target_language=None):
    audio_file = "extracted_audio.wav"
    extract_audio(video_file, audio_file)
    segments = transcribe_audio(audio_file, language)

    model, tokenizer = None, None
    if target_language is not None:
        print(f"Translating subtitles to: {target_language}")
        model_name = f"Helsinki-NLP/opus-mt-{language}-{target_language}"
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

    generate_srt(segments, output_srt, model, tokenizer)
    print(f"Subtitle file '{output_srt}' generated successfully.")

if __name__ == "__main__":
    video_file = "a.mkv"  # Replace with your video file path
    output_srt = "subtitles.srt"
    language = "ja"  # Specify the language code of the video (e.g., "ja" for Japanese)
    target_language = "en"  # Set to None for subtitles in the original language, or specify a different language code
    main(video_file, output_srt, language, target_language)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

size=  246508kB time=00:23:50.95 bitrate=1411.2kbits/s speed=1.08e+03x    
video:0kB audio:246508kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.000031%
100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 165MiB/s]
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_